## Test Bench
This  notebook is used to evaluate the performance of a compression algorithm.

It is written such that a new compression algorithm can be added without altering the bench (just change the input in the config).

In [1]:
# ADD NEW ALGORITHMS HERE
from algos.alg_wkt import Wkt
from algos.alg_wkb import Wkb
from algos.alg_meta_wkb_comp import MetaWkbComp # Wkb with Metadat
from algos.alg_meta_wkt_comp import MetaWktComp # Wkt with Metadat
from algos.alg_wkb_comp import WkbComp
from algos.alg_wkt_comp import WktComp
from algos.alg_fpd import Fpd
from algos.alg_fpd_extended import FpdExtended
from algos.alg_fpd_predictor import FpdPredictor
from algos.fpd_extended_lib.entropy_coder import *


# CONFIG
ALG = FpdExtended()
#DATASET_PATH = "data/lund_building_highway.json"
DATASET_PATH = "data/sweden_processed_whole"
#DATASET_PATH = "data/world.json"
#DATASET_PATH = "data/world_7_dec.json"
#DATASET_PATH = "data/sweden.json"
#DATASET_PATH = "data/latest_export.json"
#DATASET_PATH = "data/latest_export" # Use folder (single files)
NBR_ITER = 100000
N = 10
AVOID_DATASET_LENGTH_CHECK = True # For very large folders, disable

IS_SHP_DIR = False
#DATASET_PATH = "data/sweden-latest-free"
#DATASET_PATH = 'data/sweden-latest-free/gis_osm_buildings_a_free_1.shp'
# Used sets: ["data/sweden-latest-free/gis_osm_buildings_a_free_1.shp", "data/sweden-latest-free/gis_osm_roads_free_1.shp", "data/china-latest-free/gis_osm_water_a_free_1.shp", "data/ne_10m_admin_1_states_provinces.shp"]

COMPR_PATH = "data/testbench_compressed_single"
#COMPR_PATH = "data/testbench_compressed"

DECOMPR_PATH = "data/testbench_decompressed_single"

In [2]:
import time
import os
from enum import Enum
import pandas as pd
from datetime import datetime
import filecmp
import random
import json
import tqdm
from shapely.geometry import shape
import glob
import timeit
import shapely
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd

# List the metrics which are to be logged
ET = Enum('ET', ['COMP_TIME', 'DECOMP_TIME', 'COMP_SIZE', 'DECOMP_SIZE',
    'VERTICES', 'TYPE', 'BOUNDING_BOX', 'ADD_VERTEX', 'IS_INTERSECTING', 'INTERSECTION', 'IS_BASELINE'
]) # EVAL_TYPE

if not os.path.exists(COMPR_PATH):
   os.makedirs(COMPR_PATH)
for f in glob.glob(COMPR_PATH + '/*'):
    os.remove(f)
if not os.path.exists(DECOMPR_PATH):
   os.makedirs(DECOMPR_PATH)
for f in glob.glob(DECOMPR_PATH + '/*'):
    os.remove(f)

In [3]:
# Maps an evaluation type to the corresponding function in the algorithm.
# Note that ALG (global variable) can change, so a function is used.
from functools import reduce
import operator

def et_to_func():
    return {
        ET.VERTICES: ALG.vertices, 
        ET.TYPE: ALG.type,
        ET.BOUNDING_BOX: ALG.bounding_box,
        ET.ADD_VERTEX: ALG.add_vertex, 
        ET.IS_INTERSECTING: ALG.is_intersecting, 
        ET.INTERSECTION: ALG.intersection 
        }

# Params: "et: EVAL_TYPE, idx: geometry index to query, value_only: dont save entry in log"
def measure(et, args, value_only=False): # Works for functions taking only a geometry 'idx' as parameter
    """
    Takes an evaluation type and executes the corresponding operation, storing the execution time and resulting value.
    Also returns the resulting value of the operation.
    """
    sum_t = 0
    for _ in range(N):
        func = et_to_func()[et] # Find the function to be called
        t, v = func(args)
        if not value_only:
            sum_t += t
    
    res[et].append(sum_t / N)
    vals[et].append(v)        
    return v

SEED = 123 # If we want to enforce the same ordering and indexes for multiple runs, else None
#SEED = random.randint(0, 9999999999999999)
random.seed(SEED) # Init random

res = {x: [] for x in ['idx_unary', 'idx_binary'] + list(ET)} # Store the 'results' of the measurements
vals = {x: [] for x in ['idx_unary', 'idx_binary'] + list(ET)[4:]} # Store the 'values' for the operations, to perform correctness check later

# Currently not supporting extra entropy/chunk compression while supporting add_vertex
if cfg.USE_ENTROPY or cfg.COMPRESS_CHUNK:
    del res[ET.ADD_VERTEX]
    del vals[ET.ADD_VERTEX]
    # del res[ET.IS_INTERSECTING]
    # del res[ET.INTERSECTION]
    # del vals[ET.IS_INTERSECTING]
    # del vals[ET.INTERSECTION]
    # del res["idx_binary"]
    # del vals["idx_binary"]

# ----- LOAD DATA FROM DATA SET ------
if DATASET_PATH.endswith('.json'):
    # Extract the nested feature attribute of the geo_json file containing the geometries
    with open(DATASET_PATH, 'r') as f:
        data = json.loads(f.read())
    file_df: pd.DataFrame = pd.json_normalize(data, record_path=['features'])
    # Create a dataframe suitable for the WKT format for easy convertion to shapely objects
    df_dataset = pd.DataFrame(
        {'type': file_df['geometry.type'], 'coordinates': file_df['geometry.coordinates']})
    max_idx = len(df_dataset) - 1

    def get_shape_data(idx):
        return df_dataset.iloc[idx]
elif DATASET_PATH.endswith('.shp') or IS_SHP_DIR:
    if not DATASET_PATH.endswith('.shp') and IS_SHP_DIR:
        files = glob.glob(DATASET_PATH + '/*.shp')
    else:
        files = [DATASET_PATH]

    all_shapes = pd.DataFrame()
    for i, f in enumerate(tqdm.tqdm(files, disable=True)):
        file_df = gpd.read_file(f)
        print("Count of entries:", len(file_df))
        file_df = file_df[file_df.type != "Point"]
        print("---", DATASET_PATH, "---")
        print("Count of entries (no points):", len(file_df))
        if len(file_df) != 0:
            shapes = file_df.geometry
            all_shapes = pd.concat([all_shapes, shapes])
        
    if NBR_ITER != -1:
        all_shapes = all_shapes.sample(n=NBR_ITER, replace=True)
    
    max_idx = len(shapes) - 1

    def get_shape_data(idx):
        return shapes.iloc[idx]
else:
    if not AVOID_DATASET_LENGTH_CHECK:
        tree = os.walk(DATASET_PATH, topdown=True)
        max_idx = len(next(tree)[2]) - 1
    else:
        max_idx = 5000000

    def get_shape_data(idx):
        with open(f'{DATASET_PATH}/{idx}.json', 'r') as f:
            data = json.loads(f.read())
        
        file_df: pd.DataFrame = pd.json_normalize(data, record_path=['features'])
        # # Create a dataframe suitable for the WKT format for easy convertion to shapely objects
        df_dataset = pd.DataFrame(
             {'type': file_df['geometry.type'], 'coordinates': file_df['geometry.coordinates']})
        return df_dataset.iloc[0]
# ----- /END/ LOAD DATA FROM DATA SET ------

unary_idxs = [random.randint(0, max_idx) for i in range(NBR_ITER)] # Generate list of indexes to query on
binary_idxs = [(random.randint(0, max_idx), random.randint(0, max_idx)) for i in range(NBR_ITER)] # Generate pair of indexes to query on. # TODO: Better selection than random
random.seed(SEED) # Reset random

def act_deact_baseline(activate=True):
        cfg.BASELINE_ON = activate
        cfg.DISABLE_OPTIMIZED_INTERSECTION = False if not activate else True
        cfg.DISABLE_OPTIMIZED_ADD_VERTEX = False if not activate else True
        cfg.DISABLE_OPTIMIZED_BOUNDING_BOX = False if not activate else True

def compress(data, file_comp):
    # Compress
    if not (IS_SHP_DIR or DATASET_PATH.endswith('.shp')):
        geometry = shape(data)
    else:
        geometry = data
    t, bin = ALG.compress(geometry)

    # Write to disk
    f = open(file_comp, "wb")
    f.write(bin)
    f.close()
    return t, bin

# Compress files, benchmark unaries
for is_baseline in [True, False]:
    act_deact_baseline(is_baseline)
    for idx in tqdm.tqdm(unary_idxs): # List of single idxs
        res[ET.IS_BASELINE].append(is_baseline) 
        file_comp = f"{COMPR_PATH}/{idx}"
        #t = timeit.timeit(lambda: compress(get_shape_data(idx), file_comp), number=50)
        t, bin = compress(get_shape_data(idx), file_comp)
        res[ET.COMP_TIME].append(t) # Store delta time
        #res[ET.COMP_SIZE].append(os.stat(file_comp).st_size) # Store compressed file size
        res[ET.COMP_SIZE].append(len(bin))
    for idx in tqdm.tqdm(list(reduce(operator.concat, binary_idxs))): # List of pairs collapsed
        if idx not in unary_idxs:
            file_comp = f"{COMPR_PATH}/{idx}"
            compress(get_shape_data(idx), file_comp)

    for i in tqdm.tqdm(range(len(unary_idxs))):
    # ------------ UNARY ------------ #
        idx = unary_idxs[i]
        file_comp = f"{COMPR_PATH}/{idx}"
        file_decomp = f"{DECOMPR_PATH}/{idx}"

        res["idx_unary"].append(idx)
        vals["idx_unary"].append(idx)

        # Read binary into memory
        f = open(file_comp, "rb")
        bin = f.read()
        f.close()

        # Decompress
        t, decomp = ALG.decompress(bin)
        #decomp = shapely.to_wkt(decomp, rounding_precision=-1)
        decomp = shapely.to_wkb(decomp)
        # Write to disk
        f = open(file_decomp, "wb")
        f.write(decomp)
        f.close()
        res[ET.DECOMP_TIME].append(t) # Store delta time
        res[ET.DECOMP_SIZE].append(os.stat(file_decomp).st_size) # Store decompressed file size

        v = measure(ET.VERTICES, bin)
        measure(ET.TYPE, bin)
        measure(ET.BOUNDING_BOX, bin)
        add_idx = random.randint(0, len(v) - 2)
        add_point = (round(v[add_idx][0] + random.randint(-25, 25) * 0.00001,7), round(v[add_idx][1] + random.randint(-25, 25) * 0.00001,7))
        if not (cfg.USE_ENTROPY or cfg.COMPRESS_CHUNK):
            measure(ET.ADD_VERTEX, (bin, add_idx, add_point))

    for i in tqdm.tqdm(range(len(binary_idxs))):
    # ------------ BINARY ------------ #
        l_idx, r_idx = binary_idxs[i]

        res["idx_binary"].append(binary_idxs[i])
        vals["idx_binary"].append(binary_idxs[i])

        # Read both binary into memory
        f = open(f"{COMPR_PATH}/{l_idx}", "rb")
        l_bin = f.read()
        f.close()

        f = open(f"{COMPR_PATH}/{r_idx}", "rb")
        r_bin = f.read()
        f.close()

        measure(ET.IS_INTERSECTING, (l_bin, r_bin))
        measure(ET.INTERSECTION, (l_bin, r_bin))
df = pd.DataFrame(res)
#df

  0%|          | 0/100000 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'data/sweden_processed_whole/439257.json'

In [ ]:
df_relative = df.copy() 
def calculate_ratio(x):
    base = x[x[ET.IS_BASELINE] == True]
    non_base = x[x[ET.IS_BASELINE] == False]
   
    if not base.empty and not non_base.empty:
        return non_base.divide(base.values, axis=1)
    else:
        return None
    
df_relative = df_relative.groupby(["idx_unary", "idx_binary"]).apply(calculate_ratio)
df_relative.reset_index(inplace=True)
if 'level_2'  in  df_relative.columns:
    df_relative.drop('level_2', axis=1, inplace=True)
df_relative.drop([ET.IS_BASELINE, ET.TYPE, ET.VERTICES], axis=1, inplace=True)

df_relative = df_relative.rename(columns={ET.IS_BASELINE: "is_baseline",
                            ET.ADD_VERTEX: "Add Vertex",
                            ET.IS_INTERSECTING: "Is Intersection",
                            ET.INTERSECTION: "Intersection",
                            ET.COMP_TIME: "Compress",
                            ET.COMP_SIZE: "comp_size",
                            ET.DECOMP_TIME: "Decompress",
                            ET.DECOMP_SIZE: "decomp_size",
                            ET.BOUNDING_BOX: "Bounding Box",
                            ET.TYPE: "Type",
                            ET.VERTICES: "Verticies"})
                            


NameError: name 'df' is not defined

### Plot the Results

In [ ]:
SHOW_PREVIOUS = True
dataset_name = os.path.splitext(os.path.basename(DATASET_PATH))[0]
times = df_relative.drop(axis=1, labels=['idx_unary', 'idx_binary', "comp_size", "decomp_size"])
times['Dataset'] = dataset_name

prevs = glob.glob(f'reports/*')
for prev in prevs:
    dataset = prev.split("[")[1].split("]")[0]
    prev_df = pd.read_csv(prev)
    prev_df['Dataset'] = dataset
    prev_times = prev_df.drop(axis=1, labels=['idx_unary', 'idx_binary', 'comp_size', 'decomp_size', 'Unnamed: 0'])
    times = pd.concat([times, prev_times])

times = times.melt('Dataset', var_name="Statistic", value_name='Seconds')
times['Statistic'] = times['Statistic'].apply(lambda x: str(x))

import  plotly.express as px
times.columns = ['Dataset', 'Statistic', 'Seconds']
times = times.groupby(["Dataset",  "Statistic"]).mean()
times.reset_index(inplace=True)
times["Seconds"] *= 100
fig = px.bar(times, x="Statistic", y='Seconds', color="Dataset", barmode="group", log_y=True, text="Seconds", 
            title="Relative Execution Time for Different <br>Operations and Datasets<br>",
            category_orders={"Statistic": ["Compress", "Decompress", "Intersection", "Is Intersection", "Add Vertex", "Bounding Box"]})
fig.update_yaxes(title_text="Relative Execution time", tickfont=dict(size=16), title_font=dict(size=20))
fig.update_xaxes(title_text="Operation", title_font=dict(size=20))
#fig.update_annotations(font=dict(size=16))

fig.update_traces(texttemplate='%{text:.2f}%',textfont=dict(size=14))
fig.update_layout(legend=dict(
                            yanchor="top",
                            y=1.45,
                            xanchor="left",
                            x=0.78,
                            orientation="v",
                            font=dict(size=16),
                        ),
                  title={"font": {'size': 22}})
fig.update_xaxes(tickfont=dict(size=16))




fig.show()

### Export to CSV

In [ ]:
df_relative.to_csv("reports/" + ALG.__class__.__name__ + '_[' + dataset_name + "]_" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv")